In [1]:
import requests
import pandas as pd
from datetime import datetime, timedelta

base_url = "https://rata.digitraffic.fi/api/v1/trains"
start_date = datetime(2023, 11, 1)
end_date = datetime(2023, 11, 30)

all_train_data = []

for x in pd.date_range(start_date, end_date):
    thedate = x.strftime("%Y-%m-%d")
    url = f"{base_url}/{thedate}"
    
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        all_train_data.extend(data)
    else:
        print(f"Failed to fetch data for {thedate}")


In [4]:
import duckdb
df = pd.DataFrame(all_train_data)
con = duckdb.connect(database='../data/datalake/database.duckdb', read_only=False)


In [5]:

con.execute("drop table if exists traindata;")
con.execute("drop table if exists timetablerows;")
con.execute("drop table if exists time_rows;")
con.execute("""
create table if not exists traindata (
    trainNumber int not null,
    departureDate date,
    operatorUICCode int,
    operatorShortCode varchar,
    trainType varchar,
    trainCategory varchar,
    commuterLineID varchar,
    runningCurrently boolean,
    cancelled boolean,
    version bigint,
    timetableType varchar,
    timetableAcceptanceDate timestamp
);
""")
con.execute("""
CREATE TABLE IF NOT EXISTS timetablerows (
    trainNumber INT,
    stationShortCode VARCHAR,
    stationUICCode INT,
    countryCode VARCHAR,
    type VARCHAR,
    trainStopping BOOLEAN,
    commercialStop BOOLEAN,
    commercialTrack VARCHAR,
    cancelled BOOLEAN,
    scheduledTime TIMESTAMP,
    actualTime TIMESTAMP,
    differenceInMinutes INT
);
""")

In [6]:
time_rows = []
for x in all_train_data:
    train_n = x['trainNumber']
    for row in x['timeTableRows']:
        row['trainNumber'] = train_n
        time_rows.append(row) # datassa sisäkkäisiä sarakkeita, suoritetaan "normalisointi"
df_time = pd.DataFrame(time_rows)
df = df.drop(columns=['timeTableRows'])


In [7]:
con.begin()
df.to_sql('traindata', con, if_exists='append', index=False)
con.commit()

C:\Users\miika\AppData\Local\Temp\ipykernel_12492\610619763.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df.to_sql('traindata', con, if_exists='append', index=False)


In [8]:
con.begin()
df_time.to_sql('time_rows', con, if_exists='append', index=False)
con.execute("COPY time_rows TO 'time_rows.csv' (HEADER, DELIMITER ',');") # csv tiedostoksi
con.commit()

C:\Users\miika\AppData\Local\Temp\ipykernel_12492\4049882935.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_time.to_sql('time_rows', con, if_exists='append', index=False)


In [9]:
con.close()